# Week #2 Case Study: RxMix-a-Lot

This Jupyter notebook will demonstrate how to perform the essential steps of the case study with Python using the relevant application programming interfaces (APIs).

## Import libraries/modules

In [63]:
import requests
import pandas as pd
import time

## Define functions

In [64]:
# Define functions for translating between concepts

def get_rxcui_from_ndc(ndc):
    url = 'https://rxnav.nlm.nih.gov/REST/rxcui.json'
    params = {
        'idtype': 'NDC',
        'id': ndc,
        'allsrc': '0'
    }
    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        return data.get('idGroup', {}).get('rxnormId', [None])[0]
    except requests.RequestException as e:
        print(f"Failed to get RxCUI for NDC {ndc}: {e}")
        return None

def get_cvx_from_rxcui(rxcui):
    """Get CVX code for an RxCUI by checking its relationships"""
    # First get ingredient information
    url = f'https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/property.json'
    params = {
        'propName': 'CVX'
    }
    try:
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()
        return data.get('propConceptGroup', {}) \
            .get('propConcept', [{}])[0] \
            .get('propValue')
    except:
        None

def build_ndc_rxcui_cvx_crosswalk(ndc_list):
    records = []
    for ndc in ndc_list:
        rxcui = get_rxcui_from_ndc(ndc)
        cvx = get_cvx_from_rxcui(rxcui) if rxcui else None
        records.append({
            'NDC': ndc,
            'RxCUI': rxcui,
            'CVX': cvx
        })
        time.sleep(0.2)  # Gentle pause to avoid rate limits
    return pd.DataFrame(records)

## Import the NDC data from the flat file

In [65]:
# Get the data from 'Tetanus NDC Batch.txt'
# The strip method removes whitespace characters such as the \n that
# separates each line in the file.
with open('Tetanus NDC Batch.txt', 'r') as file:
    ndc_list = [ndc.strip() for ndc in file]

# Print the list of NDCs that we are starting with.
print(ndc_list)

['13533-131-00', '14362-0111-3', '49281-215-88', '50090-2062-1', '50090-1377-2', '58160-842-05']


## Build a dataframe that shows the crosswalk between terms

In [66]:
# Build the crosswalk
df_crosswalk = build_ndc_rxcui_cvx_crosswalk(ndc_list)

# Print the dataframe
print(df_crosswalk)

            NDC    RxCUI  CVX
0  13533-131-00  2106334   09
1  14362-0111-3  2106334   09
2  49281-215-88  1190919  113
3  50090-2062-1  1190919  113
4  50090-1377-2  1300370  115
5  58160-842-05  1300370  115
